# Load data from local SCV files

In [ ]:
# Load from local files
import pandas as pd
import numpy as np
import glob

# Path to csv files
covid_data = 'Data/covid/*.csv'
vaccination_usa = 'Data/vaccinations/us.csv'
vaccination_usa_state = 'Data/vaccinations/us_state_vaccinations.csv'


def read_multiple_csv(regex_path):
    data = pd.DataFrame()
    # Read all css files per day comnine in one dataset
    for file in glob.glob(covid_data):
        df = pd.read_csv(file)
        # Check if "Last_update" column has null values, fill it with forward fill
        # propagate[s] last valid observation forward to next valid
        if df['Last_Update'].isna().sum() > 0:
            df.fillna(method='ffill', inplace=True)
        # Remove invalid states from dataset
        df = df[~df['Province_State'].isin(['Diamond Princess', 'Grand Princess', 'Recovered'])]
        # Set Recovered field to 0 when no confirmed cases
        df['Recovered'] = df.apply(lambda x: x['Confirmed'] if x['Confirmed']==0 else x['Recovered'], axis=1)
        data = data.append(df,ignore_index=True)
    return data


# Load data
vac_usa_data = pd.read_csv(vaccination_usa)
vac_state_data = pd.read_csv(vaccination_usa_state)
covid_data = read_multiple_csv(covid_data)

# Load SCV files from remote github repository

In [9]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
import re

# Example of github link
# https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/us_state_vaccinations.csv

# Hopkins covid 19 dataset
git_raw_domain = 'https://raw.githubusercontent.com/'
covid_url = "https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us"

# URL Link to cvs file for vaccination in USA by day
url_vaccination_usa = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/'\
                       + 'country_data/United%20States.csv'
# URL Link to cvs file for vaccination in USA by state
url_vaccination_state = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/'\
                       + 'us_state_vaccinations.csv'


def load_file_from_github(url, raw_domain):
    """
        Load file from github:
            url - github url for all csv files
            raw_domain  - domain to get raw csv file
    """
    # Get github page
    txt = urlopen(url).read().decode('utf8')
    links = re.findall("href=[\"\'](.*?csv)[\"\']", txt)
    # Create pandas DataFrame
    data = pd.DataFrame()

    for link in links:
        df = pd.read_csv(raw_domain + link.replace('blob/', ''),error_bad_lines=False)
        # Check if "Last_update" column has null values, fill it with forward fill
        # propagate[s] last valid observation forward to next valid
        if df['Last_Update'].isna().sum() > 0:
            # Each file should be for one specific day
            df.fillna(method='ffill', inplace=True)
        # Remove invalid states from dataset
        df = df[~df['Province_State'].isin(['Diamond Princess', 'Grand Princess', 'Recovered'])]
        # Set Recovered field to 0 when no confirmed cases
        df['Recovered'] = df.apply(lambda x: x['Confirmed'] if x['Confirmed']==0 else x['Recovered'], axis=1)
        data = data.append(df,ignore_index=True)
    return data

# Load data
vac_usa_data = pd.read_csv(url_vaccination_usa,error_bad_lines=False)
vac_state_data = pd.read_csv(url_vaccination_state,error_bad_lines=False)
covid_data = load_file_from_github(covid_url, git_raw_domain)

In [20]:
# Sort dataset by date
vac_usa_data = vac_usa_data.sort_values(by='date')
vac_usa_data

,location,date,vaccine,source_url,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,United States,2020-12-20,Pfizer/BioNTech,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,556208,556208.0,NaN
1,United States,2020-12-21,Pfizer/BioNTech,https://covid.cdc.gov/covid-data-tracker/#vacc...,614117,614117.0,NaN
2,United States,2020-12-23,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,1008025,1008025.0,NaN
3,United States,2020-12-26,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,1944585,1944585.0,NaN
4,United States,2020-12-28,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,2127143,2127143.0,NaN
5,United States,2020-12-30,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,2794588,2794588.0,NaN
6,United States,2021-01-02,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4225756,4225756.0,NaN
7,United States,2021-01-04,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4563260,4563260.0,NaN
8,United States,2021-01-05,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4836469,4836469.0,NaN
9,United States,2021-01-06,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,5306797,5306797.0,NaN


In [19]:
# Sort dataset by date
vac_state_dat = vac_state_data.sort_values(by='date')
vac_state_dat

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
2090,2020-12-20,United States,556208.0,NaN,NaN,NaN,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091,2020-12-21,United States,614117.0,NaN,NaN,NaN,0.18,NaN,NaN,NaN,57909.000000,57909.0,174.0,NaN
2092,2020-12-22,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196954.000000,127432.0,384.0,NaN
2093,2020-12-23,United States,1008025.0,NaN,NaN,NaN,0.30,NaN,NaN,NaN,196954.000000,150606.0,454.0,NaN
2094,2020-12-24,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,312186.666667,191001.0,575.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,2021-02-18,Louisiana,795511.0,928425.0,533933.0,5.59,17.11,259851.0,11.48,19.97,3937.000000,22356.0,4809.0,0.857
1975,2021-02-18,South Carolina,778113.0,939350.0,562999.0,3.92,15.11,201703.0,10.94,18.24,22691.000000,21374.0,4151.0,0.828
2416,2021-02-18,West Virginia,403532.0,441500.0,254778.0,8.30,22.52,148742.0,14.22,24.64,7523.000000,8965.0,5002.0,0.914
1139,2021-02-18,Massachusetts,1214821.0,1526150.0,912258.0,4.35,17.62,299871.0,13.24,22.14,44423.000000,41458.0,6015.0,0.796


In [28]:
# Sort dataset by date
covid_data = covid_data.sort_values(by=['Last_Update', 'Province_State'])
covid_data

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,UID,ISO3,Testing_Rate,People_Tested,Mortality_Rate
2744,Alabama,US,2020-04-12,32.3182,-86.9023,3667,93,91214.076271,3470.0,1.0,75.988020,84000001.0,USA,460.300152,21583.0,2.610160
2745,Alaska,US,2020-04-12,61.3707,-152.4044,272,8,66.000000,264.0,2.0,45.504049,84000002.0,USA,1344.711576,8038.0,2.941176
2797,American Samoa,US,2020-04-12,-14.2710,-170.1322,0,0,0.000000,0.0,60.0,0.000000,16.0,ASM,5.391708,3.0,0.000000
2746,Arizona,US,2020-04-12,33.7298,-111.4312,3542,115,66.000000,3427.0,4.0,48.662422,84000004.0,USA,578.522286,42109.0,3.246753
2747,Arkansas,US,2020-04-12,34.9697,-92.3731,1280,27,367.000000,1253.0,5.0,49.439423,84000005.0,USA,761.753354,19722.0,2.109375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,Virginia,US,2021-02-19,37.7693,-78.1700,557896,7090,45305.000000,505501.0,51.0,6536.169622,84000051.0,USA,66543.510711,2787106.0,1.928971
2740,Washington,US,2021-02-19,47.4009,-121.4905,332007,4803,0.000000,327204.0,53.0,4359.969339,84000053.0,USA,65631.125743,2592766.0,2.078735
2741,West Virginia,US,2021-02-19,38.4912,-80.9545,128760,2236,116436.000000,10088.0,54.0,7184.678489,84000054.0,USA,118076.530552,849461.0,1.774398
2742,Wisconsin,US,2021-02-19,44.2685,-89.6165,610055,6816,541515.000000,61724.0,55.0,10477.662778,84000055.0,USA,113344.195915,3564737.0,0.864590


In [108]:
covid_data.rename(columns = {'Province_State':'location', 'Last_Update': 'date' }, inplace = True)
covid_data

,location,Country_Region,date,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,UID,ISO3,Testing_Rate,People_Tested,Mortality_Rate
2744,Alabama,US,2020-04-12,32.3182,-86.9023,3667,93,91214.076271,3470.0,1.0,75.988020,84000001.0,USA,460.300152,21583.0,2.610160
2745,Alaska,US,2020-04-12,61.3707,-152.4044,272,8,66.000000,264.0,2.0,45.504049,84000002.0,USA,1344.711576,8038.0,2.941176
2797,American Samoa,US,2020-04-12,-14.2710,-170.1322,0,0,0.000000,0.0,60.0,0.000000,16.0,ASM,5.391708,3.0,0.000000
2746,Arizona,US,2020-04-12,33.7298,-111.4312,3542,115,66.000000,3427.0,4.0,48.662422,84000004.0,USA,578.522286,42109.0,3.246753
2747,Arkansas,US,2020-04-12,34.9697,-92.3731,1280,27,367.000000,1253.0,5.0,49.439423,84000005.0,USA,761.753354,19722.0,2.109375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,Virginia,US,2021-02-19,37.7693,-78.1700,557896,7090,45305.000000,505501.0,51.0,6536.169622,84000051.0,USA,66543.510711,2787106.0,1.928971
2740,Washington,US,2021-02-19,47.4009,-121.4905,332007,4803,0.000000,327204.0,53.0,4359.969339,84000053.0,USA,65631.125743,2592766.0,2.078735
2741,West Virginia,US,2021-02-19,38.4912,-80.9545,128760,2236,116436.000000,10088.0,54.0,7184.678489,84000054.0,USA,118076.530552,849461.0,1.774398
2742,Wisconsin,US,2021-02-19,44.2685,-89.6165,610055,6816,541515.000000,61724.0,55.0,10477.662778,84000055.0,USA,113344.195915,3564737.0,0.864590


## Handle NaN values in COVID dataset 

In [131]:
# Convert data to YYYY-MM-DD format
covid_data['date'] = pd.to_datetime(covid_data['date'], format='%Y/%m/%d %H:%M:%S').dt.strftime('%Y-%m-%d')
covid_data

,location,Country_Region,date,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,UID,ISO3,Testing_Rate,People_Tested,Mortality_Rate
2744,Alabama,US,2020-04-12,32.3182,-86.9023,3667,93,91214.076271,3470.0,1.0,75.988020,84000001.0,USA,460.300152,21583.0,2.610160
2745,Alaska,US,2020-04-12,61.3707,-152.4044,272,8,66.000000,264.0,2.0,45.504049,84000002.0,USA,1344.711576,8038.0,2.941176
2797,American Samoa,US,2020-04-12,-14.2710,-170.1322,0,0,0.000000,0.0,60.0,0.000000,16.0,ASM,5.391708,3.0,0.000000
2746,Arizona,US,2020-04-12,33.7298,-111.4312,3542,115,66.000000,3427.0,4.0,48.662422,84000004.0,USA,578.522286,42109.0,3.246753
2747,Arkansas,US,2020-04-12,34.9697,-92.3731,1280,27,367.000000,1253.0,5.0,49.439423,84000005.0,USA,761.753354,19722.0,2.109375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,Virginia,US,2021-02-19,37.7693,-78.1700,557896,7090,45305.000000,505501.0,51.0,6536.169622,84000051.0,USA,66543.510711,2787106.0,1.928971
2740,Washington,US,2021-02-19,47.4009,-121.4905,332007,4803,0.000000,327204.0,53.0,4359.969339,84000053.0,USA,65631.125743,2592766.0,2.078735
2741,West Virginia,US,2021-02-19,38.4912,-80.9545,128760,2236,116436.000000,10088.0,54.0,7184.678489,84000054.0,USA,118076.530552,849461.0,1.774398
2742,Wisconsin,US,2021-02-19,44.2685,-89.6165,610055,6816,541515.000000,61724.0,55.0,10477.662778,84000055.0,USA,113344.195915,3564737.0,0.864590


In [30]:
# Find all missing values
covid_data.isna().sum()

Province_State    0
Country_Region    0
Last_Update       0
Lat               0
Long_             0
Confirmed         0
Deaths            0
Recovered         0
Active            0
FIPS              0
Incident_Rate     0
UID               0
ISO3              0
Testing_Rate      0
People_Tested     0
Mortality_Rate    0
dtype: int64

In [23]:
# Drop columns that have more than 60-70% of missing values
# 17360 rows / 11915 = 0.68
covid_data.drop(['People_Hospitalized', 'Hospitalization_Rate', 
                'Total_Test_Results', 'Case_Fatality_Ratio'], axis=1, inplace=True)

In [24]:
covid_data.isna().sum()

Province_State       0
Country_Region       0
Last_Update          0
Lat                  0
Long_                0
Confirmed            0
Deaths               0
Recovered         2525
Active               9
FIPS                 0
Incident_Rate        0
UID                  0
ISO3                 0
Testing_Rate         0
People_Tested     5712
Mortality_Rate    5914
dtype: int64

In [132]:
# Create tmp DataFrame with missing values and States
df_nan = covid_data[['location','Recovered', 'People_Tested', 'Mortality_Rate', 'Active']]
# Group by State and use interpolate method to fill out missing values with previous value for the same state
df_interpolated = df_nan.groupby('location').apply(lambda x: x.interpolate(method='linear'))
# Check that missing value amount is drastically reduced
df_interpolated.isna().sum()

location          0
Recovered         0
People_Tested     0
Mortality_Rate    0
Active            0
dtype: int64

In [26]:
df_interpolated

,Province_State,Recovered,People_Tested,Mortality_Rate,Active
2785,Rhode Island,35.0,20350.0,2.363977,2602.0
2773,Nevada,169.0,24611.0,3.949224,2724.0
2774,New Hampshire,236.0,10925.0,2.475780,906.0
2775,New Jersey,236.0,126735.0,3.799515,59500.0
2776,New Mexico,235.0,28692.0,2.088353,1219.0
...,...,...,...,...,...
2711,Massachusetts,477796.0,6720526.0,5.921271,70494.0
2712,Michigan,517991.0,5213961.0,3.468327,99731.0
2713,Minnesota,463041.0,3077966.0,1.498380,6779.0
2715,Missouri,397080.0,2654691.0,1.486830,481766.0


In [133]:
# Group by state and  fill out missing values with mean value 
df_means = df_interpolated.groupby('location').transform(lambda x:x.fillna(x.mean()))
# Update original DataFrame with new values
covid_data.update(df_means)
covid_data.isna().sum()

location          0
Country_Region    0
date              0
Lat               0
Long_             0
Confirmed         0
Deaths            0
Recovered         0
Active            0
FIPS              0
Incident_Rate     0
UID               0
ISO3              0
Testing_Rate      0
People_Tested     0
Mortality_Rate    0
dtype: int64

In [31]:
covid_data

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,UID,ISO3,Testing_Rate,People_Tested,Mortality_Rate
2744,Alabama,US,2020-04-12,32.3182,-86.9023,3667,93,91214.076271,3470.0,1.0,75.988020,84000001.0,USA,460.300152,21583.0,2.610160
2745,Alaska,US,2020-04-12,61.3707,-152.4044,272,8,66.000000,264.0,2.0,45.504049,84000002.0,USA,1344.711576,8038.0,2.941176
2797,American Samoa,US,2020-04-12,-14.2710,-170.1322,0,0,0.000000,0.0,60.0,0.000000,16.0,ASM,5.391708,3.0,0.000000
2746,Arizona,US,2020-04-12,33.7298,-111.4312,3542,115,66.000000,3427.0,4.0,48.662422,84000004.0,USA,578.522286,42109.0,3.246753
2747,Arkansas,US,2020-04-12,34.9697,-92.3731,1280,27,367.000000,1253.0,5.0,49.439423,84000005.0,USA,761.753354,19722.0,2.109375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,Virginia,US,2021-02-19,37.7693,-78.1700,557896,7090,45305.000000,505501.0,51.0,6536.169622,84000051.0,USA,66543.510711,2787106.0,1.928971
2740,Washington,US,2021-02-19,47.4009,-121.4905,332007,4803,0.000000,327204.0,53.0,4359.969339,84000053.0,USA,65631.125743,2592766.0,2.078735
2741,West Virginia,US,2021-02-19,38.4912,-80.9545,128760,2236,116436.000000,10088.0,54.0,7184.678489,84000054.0,USA,118076.530552,849461.0,1.774398
2742,Wisconsin,US,2021-02-19,44.2685,-89.6165,610055,6816,541515.000000,61724.0,55.0,10477.662778,84000055.0,USA,113344.195915,3564737.0,0.864590


## Handle NaN values in Vaccination dataset in USA

In [32]:
vac_usa_data

,location,date,vaccine,source_url,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,United States,2020-12-20,Pfizer/BioNTech,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,556208,556208.0,NaN
1,United States,2020-12-21,Pfizer/BioNTech,https://covid.cdc.gov/covid-data-tracker/#vacc...,614117,614117.0,NaN
2,United States,2020-12-23,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,1008025,1008025.0,NaN
3,United States,2020-12-26,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,1944585,1944585.0,NaN
4,United States,2020-12-28,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,2127143,2127143.0,NaN
5,United States,2020-12-30,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,2794588,2794588.0,NaN
6,United States,2021-01-02,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4225756,4225756.0,NaN
7,United States,2021-01-04,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4563260,4563260.0,NaN
8,United States,2021-01-05,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4836469,4836469.0,NaN
9,United States,2021-01-06,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,5306797,5306797.0,NaN


In [33]:
vac_usa_data.isna().sum()

location                    0
date                        0
vaccine                     0
source_url                  0
total_vaccinations          0
people_vaccinated           1
people_fully_vaccinated    15
dtype: int64

In [34]:
# Fill missing value with mean value in people_vaccinated column
vac_usa_data['people_vaccinated'].fillna((vac_usa_data['people_vaccinated'].mean()), inplace=True)
vac_usa_data.isna().sum()

location                    0
date                        0
vaccine                     0
source_url                  0
total_vaccinations          0
people_vaccinated           0
people_fully_vaccinated    15
dtype: int64

In [35]:
vac_usa_data['people_fully_vaccinated'].fillna(0, inplace=True)
vac_usa_data.isna().sum()

location                   0
date                       0
vaccine                    0
source_url                 0
total_vaccinations         0
people_vaccinated          0
people_fully_vaccinated    0
dtype: int64

## Handle NaN values in Vaccination dataset in USA by state

In [36]:
vac_state_data

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.52,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.27,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,2021-02-14,Wyoming,99099.0,122200.0,71653.0,4.69,17.12,27132.0,12.38,21.11,9961.0,3892.0,6725.0,0.811
2489,2021-02-15,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,543.5,3312.0,5723.0,NaN
2490,2021-02-16,Wyoming,100186.0,122200.0,72339.0,4.76,17.31,27531.0,12.50,21.11,543.5,3390.0,5857.0,0.820
2491,2021-02-17,Wyoming,100186.0,127075.0,72339.0,4.76,17.31,27531.0,12.50,21.96,0.0,2953.0,5102.0,0.788


In [37]:
vac_state_data.isna().sum()

date                                     0
location                                 0
total_vaccinations                     270
total_distributed                      314
people_vaccinated                      326
people_fully_vaccinated_per_hundred    540
total_vaccinations_per_hundred         438
people_fully_vaccinated                408
people_vaccinated_per_hundred          463
distributed_per_hundred                451
daily_vaccinations_raw                  65
daily_vaccinations                      65
daily_vaccinations_per_million         248
share_doses_used                       314
dtype: int64

In [105]:
# Find all unique states in vac_state_data dataset
vac_states = vac_state_data['location'].unique()
vac_states

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
       'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota',
       'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming'], dtype=object)

In [134]:
# Find all unique states in covid_date dataset
data_states = covid_data['location'].unique()
data_states 

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virgin Islands', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [135]:
# Find difference between two tables 
np.setdiff1d(vac_states, data_states)

array([], dtype=object)

In [44]:
# Drop island's data
vac_state_data = vac_state_data[~vac_state_data['location'].isin(['Federated States of Micronesia', 'Marshall Islands', 'Republic of Palau'])]
# Change "New York State" to "New York"
vac_state_data['location'].mask(vac_state_data['location'] == 'New York State', 'New York', inplace=True)

In [100]:
# Aggregate Indian Health Svc and Indiana
tmp = vac_state_data[vac_state_data['location'].isin(['Indian Health Svc', 'Indiana'])]
tmp = tmp.groupby('date', as_index=False).agg({ 'total_vaccinations': np.sum,
                          'total_distributed': np.sum,
                          'people_vaccinated': np.sum,
                          'people_fully_vaccinated_per_hundred': np.mean,
                          'total_vaccinations_per_hundred': np.mean,
                          'people_fully_vaccinated': np.sum, 
                          'people_vaccinated_per_hundred': np.mean,
                          'distributed_per_hundred': np.mean,
                          'daily_vaccinations_raw': np.sum,
                          'daily_vaccinations': np.sum,
                          'daily_vaccinations_per_million': np.mean,
                          'share_doses_used': np.mean,
                                                    
})
# Add location column
tmp['location'] = 'Indiana'
vac_state_data.drop(vac_state_data[vac_state_data['location'].isin(['Indian Health Svc', 'Indiana'])].index, inplace = True)
vac_state_data.append(tmp)

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
2090,2020-12-20,United States,556208.0,NaN,NaN,NaN,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091,2020-12-21,United States,614117.0,NaN,NaN,NaN,0.18,NaN,NaN,NaN,57909.000000,57909.0,174.0,NaN
2092,2020-12-22,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196954.000000,127432.0,384.0,NaN
2093,2020-12-23,United States,1008025.0,NaN,NaN,NaN,0.30,NaN,NaN,NaN,196954.000000,150606.0,454.0,NaN
2094,2020-12-24,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,312186.666667,191001.0,575.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2021-02-14,Indiana,378614.0,695550.0,273379.0,NaN,NaN,99411.0,NaN,NaN,15812.000000,13796.0,NaN,0.544
34,2021-02-15,Indiana,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,3335.000000,12944.0,NaN,NaN
35,2021-02-16,Indiana,385284.0,697025.0,278647.0,NaN,NaN,100756.0,NaN,NaN,3335.000000,12558.0,NaN,0.553
36,2021-02-17,Indiana,390833.0,697025.0,282359.0,5.04,19.22,102592.0,13.88,34.27,5549.000000,12017.0,5909.0,0.561


In [101]:
tmp = vac_state_data[vac_state_data['location'].isin(['Bureau of Prisons', 'Dept of Defense', 'Long Term Care',
       'United States', 'Veterans Health', 'District of Columbia'])]
tmp = tmp.groupby('date', as_index=False).agg({ 'total_vaccinations': np.sum,
                          'total_distributed': np.sum,
                          'people_vaccinated': np.sum,
                          'people_fully_vaccinated_per_hundred': np.mean,
                          'total_vaccinations_per_hundred': np.mean,
                          'people_fully_vaccinated': np.sum, 
                          'people_vaccinated_per_hundred': np.mean,
                          'distributed_per_hundred': np.mean,
                          'daily_vaccinations_raw': np.sum,
                          'daily_vaccinations': np.sum,
                          'daily_vaccinations_per_million': np.mean,
                          'share_doses_used': np.mean,
                                                    
})
# Add location column
tmp['location'] = 'District of Columbia'
vac_state_data.drop(vac_state_data[vac_state_data['location'].isin(['Bureau of Prisons', 'Dept of Defense', 'Long Term Care',
       'United States', 'Veterans Health', 'District of Columbia'])].index, inplace = True)
vac_state_data.append(tmp)

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,NaN,NaN,NaN,0.20700
38,2021-01-12,Alaska,35838.0,141600.0,22486.0,0.74,4.90,5400.0,3.07,19.36,NaN,NaN,NaN,0.25300
76,2021-01-12,American Samoa,2124.0,10650.0,842.0,0.47,3.81,260.0,1.51,19.12,NaN,NaN,NaN,0.19900
114,2021-01-12,Arizona,141355.0,563025.0,95141.0,0.12,1.94,8343.0,1.31,7.74,NaN,NaN,NaN,0.25100
152,2021-01-12,Arkansas,40879.0,274400.0,39357.0,0.00,1.36,8.0,1.30,9.09,NaN,NaN,NaN,0.14900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2021-02-14,District of Columbia,61107367.0,73229025.0,40031031.0,4.24,15.93,14727164.0,11.53,21.10,2522467.0,1882165.0,5023.0,0.82975
57,2021-02-15,District of Columbia,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,1288633.0,1871383.0,5006.0,NaN
58,2021-02-16,District of Columbia,63684633.0,74847475.0,41477036.0,4.52,16.63,15693016.0,11.95,21.58,1288633.0,1930896.0,5170.0,0.84025
59,2021-02-17,District of Columbia,64880193.0,75651625.0,42096002.0,4.66,16.95,16159902.0,12.13,21.81,1195560.0,1844390.0,4954.0,0.83900


In [102]:
# Group by location and use interpolate method to fill out missing values with previous value for the same state
df_interpolated = vac_state_data.groupby('location').apply(lambda x: x.interpolate(method='linear'))
# Check that missing value amount is drastically reduced
df_interpolated.isna().sum()

date                                    0
location                                0
total_vaccinations                      0
total_distributed                       0
people_vaccinated                      11
people_fully_vaccinated_per_hundred    32
total_vaccinations_per_hundred          0
people_fully_vaccinated                32
people_vaccinated_per_hundred          11
distributed_per_hundred                 0
daily_vaccinations_raw                 54
daily_vaccinations                     54
daily_vaccinations_per_million         54
share_doses_used                        0
dtype: int64

In [103]:
# Group by state and  fill out missing values with mean value 
df_means = df_interpolated.groupby('location').transform(lambda x:x.fillna(x.mean()))
# Update original DataFrame with new values
vac_state_data.update(df_means)
vac_state_data.isna().sum()

date                                   0
location                               0
total_vaccinations                     0
total_distributed                      0
people_vaccinated                      0
people_fully_vaccinated_per_hundred    0
total_vaccinations_per_hundred         0
people_fully_vaccinated                0
people_vaccinated_per_hundred          0
distributed_per_hundred                0
daily_vaccinations_raw                 0
daily_vaccinations                     0
daily_vaccinations_per_million         0
share_doses_used                       0
dtype: int64

In [104]:
vac_state_data

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.44,7.69,16051.459459,15314.972973,3123.459459,0.207
38,2021-01-12,Alaska,35838.0,141600.0,22486.0,0.74,4.90,5400.0,3.07,19.36,4413.864865,4341.783784,5935.135135,0.253
76,2021-01-12,American Samoa,2124.0,10650.0,842.0,0.47,3.81,260.0,1.51,19.12,382.297297,406.378378,7297.297297,0.199
114,2021-01-12,Arizona,141355.0,563025.0,95141.0,0.12,1.94,8343.0,1.31,7.74,28686.891892,27487.945946,3776.459459,0.251
152,2021-01-12,Arkansas,40879.0,274400.0,39357.0,0.00,1.36,8.0,1.30,9.09,12158.324324,14440.216216,4785.027027,0.149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2340,2021-02-18,Virginia,1503786.0,1714800.0,1076555.0,4.64,17.62,396059.0,12.61,20.09,43683.000000,40084.000000,4696.000000,0.877
2378,2021-02-18,Washington,1255764.0,1500325.0,914138.0,4.30,16.49,327192.0,12.00,19.70,29844.000000,30817.000000,4047.000000,0.837
2416,2021-02-18,West Virginia,403532.0,441500.0,254778.0,8.30,22.52,148742.0,14.22,24.64,7523.000000,8965.000000,5002.000000,0.914
2454,2021-02-18,Wisconsin,1040145.0,1243125.0,762117.0,4.57,17.86,266163.0,13.09,21.35,27654.000000,30553.000000,5247.000000,0.837


In [126]:
# Merge two Dataframes together
complete_df = pd.merge(covid_data,
                       vac_state_data,
                       on=['date', 'location'], 
                       how='left')
complete_df

,location,Country_Region,date,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,Alabama,US,2020-04-12,32.3182,-86.9023,3667,93,91214.076271,3470.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,US,2020-04-12,61.3707,-152.4044,272,8,66.000000,264.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,American Samoa,US,2020-04-12,-14.2710,-170.1322,0,0,0.000000,0.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona,US,2020-04-12,33.7298,-111.4312,3542,115,66.000000,3427.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas,US,2020-04-12,34.9697,-92.3731,1280,27,367.000000,1253.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,Virginia,US,2021-02-19,37.7693,-78.1700,557896,7090,45305.000000,505501.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17524,Washington,US,2021-02-19,47.4009,-121.4905,332007,4803,0.000000,327204.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17525,West Virginia,US,2021-02-19,38.4912,-80.9545,128760,2236,116436.000000,10088.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17526,Wisconsin,US,2021-02-19,44.2685,-89.6165,610055,6816,541515.000000,61724.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# Forward prapogate values
complete_df = complete_df.groupby('location').apply(lambda x: x.interpolate(method='linear'))
# Fill out all missing values with 0
complete_df.fillna(0, inplace =True)
complete_df[complete_df['location'] == 'Wyoming'].iloc[-50:]

,location,Country_Region,date,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
14783,Wyoming,US,2021-01-01,42.756,-107.3025,44409,438,42570.0,1401.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
14839,Wyoming,US,2021-01-02,42.756,-107.3025,44409,438,42570.0,1401.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
14895,Wyoming,US,2021-01-03,42.756,-107.3025,44573,438,43037.0,1098.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
14951,Wyoming,US,2021-01-04,42.756,-107.3025,44875,438,43068.0,1369.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
15007,Wyoming,US,2021-01-05,42.756,-107.3025,45257,438,43420.0,1399.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
15063,Wyoming,US,2021-01-06,42.756,-107.3025,45569,464,43563.0,1542.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
15119,Wyoming,US,2021-01-07,42.756,-107.3025,45890,464,43642.0,1784.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
15175,Wyoming,US,2021-01-08,42.756,-107.3025,46168,489,43949.0,1730.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
15231,Wyoming,US,2021-01-09,42.756,-107.3025,46647,489,44279.0,1879.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000
15287,Wyoming,US,2021-01-10,42.756,-107.3025,46719,489,44490.0,1740.0,56.0,...,0.00,0.000,0.000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0000


In [125]:
# Safe pre-processed dataset in CSV file locally
complete_df.to_csv('processed_by_sate.csv', encoding='utf-8', index=False)